## Подключение к чистой базе данных

In [ ]:
!pip install pymysql
!pip install pyngrok
!pip install skyfield

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.2/336.2 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.3/232.3 kB 17.0 MB/s eta 0:00:00


## Функциональная часть все все действующие функции для предсказания прогноза цен на акции

## Дополнительно сюда встовил запуск базы данных для автоматизации запуска скрипта

In [ ]:
import pymysql
from sqlalchemy import create_engine
from skyfield.api import load
from datetime import datetime, timedelta
import pytz
import pandas as pd
import requests
from pyngrok import ngrok
from http.server import BaseHTTPRequestHandler, HTTPServer
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import json

db_config = {
    "host": "sql7.freemysqlhosting.net",
    "user": "sql7755157",
    "password": "EpJR55HqCE",
    "database": "sql7755157",
    "port": 3306
}

DB_SERVER = "sql7.freemysqlhosting.net"
DB_NAME = "sql7755157"
DB_USER = "sql7755157"
DB_PASSWORD = "EpJR55HqCE"
DB_PORT = 3306



BASE_URL = "https://iss.moex.com/iss/history/engines/stock/markets/shares/securities"

def fetch_first_trading_day(company):

    try:
        response = requests.get(f"{BASE_URL}/{company}.json", params={"iss.meta": "off", "lang": "ru"})
        response.raise_for_status()

        data = response.json()
        history_data = data.get('history', {}).get('data', [])
        history_columns = data.get('history', {}).get('columns', [])

        df = pd.DataFrame(history_data, columns=history_columns)

        if "TRADEDATE" in df.columns:
            return df["TRADEDATE"].min()
        else:
            raise ValueError("TRADEDATE колонка не найдена в данных.")
    except Exception as e:
        raise RuntimeError(f"Ошибка при запросе первого дня торгов: {e}")

def fetch_historical_data(company, start_date):

    all_data = []
    start = 0
    try:
        while True:

            params = {
                "from": start_date,
                "iss.meta": "off",
                "start": start,
                "lang": "ru"
            }

            response = requests.get(f"{BASE_URL}/{company}.json", params=params)
            response.raise_for_status()

            data = response.json()
            history_data = data.get('history', {}).get('data', [])
            history_columns = data.get('history', {}).get('columns', [])

            if not history_data:
                break

            df = pd.DataFrame(history_data, columns=history_columns)
            all_data.append(df)

            start += 100

        full_data = pd.concat(all_data, ignore_index=True)

        if "TRADEDATE" in full_data.columns and "CLOSE" in full_data.columns:
            result_df = full_data[["TRADEDATE", "CLOSE"]].rename(columns={
                "TRADEDATE": "date",
                "CLOSE": "value"
            })
            return result_df
        else:
            raise ValueError("Не найдены необходимые колонки (TRADEDATE и CLOSE).")
    except Exception as e:
        raise RuntimeError(f"Ошибка при запросе исторических данных: {e}")

def company_stock_history(company):

    try:
        first_day = fetch_first_trading_day(company)
        historical_data = fetch_historical_data(company, first_day)
        return historical_data
    except Exception as e:
        print(f"Ошибка: {e}")
        return None

def merge_stock_with_astro(stock_data):

    if 'date' not in stock_data.columns:
        raise ValueError("Датасет 'stock_data' должен содержать столбец 'date'.")
    engine = create_engine(f"mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_SERVER}:{DB_PORT}/{DB_NAME}")

    start_date = stock_data['date'].min()
    end_date = stock_data['date'].max()

    query = f"""
    SELECT *,
           CONCAT(year, '-', LPAD(month, 2, '0'), '-', LPAD(day, 2, '0')) AS full_date
    FROM astro
    WHERE CONCAT(year, '-', LPAD(month, 2, '0'), '-', LPAD(day, 2, '0')) BETWEEN '{start_date}' AND '{end_date}';
    """

    astro_data = pd.read_sql(query, engine)

    astro_data['full_date'] = pd.to_datetime(astro_data['full_date'])

    stock_data['date'] = pd.to_datetime(stock_data['date'])

    eph_stock_data = pd.merge(stock_data, astro_data, left_on='date', right_on='full_date', how='left')

    eph_stock_data = eph_stock_data.drop(columns=['full_date'])

    return eph_stock_data

def prepare_and_train_model(eph_stock_data, target_column='value', timesteps=30, epochs=20, batch_size=32):

    eph_stock_data = eph_stock_data.dropna()

    eph_stock_data = eph_stock_data.drop(columns=['date_x', 'date_y'], errors='ignore')

    scaler = StandardScaler()
    feature_columns = eph_stock_data.columns.difference([target_column])
    eph_stock_data[feature_columns] = scaler.fit_transform(eph_stock_data[feature_columns])

    X, y = [], []
    for i in range(len(eph_stock_data) - timesteps):

        X.append(eph_stock_data.iloc[i:i+timesteps][feature_columns].values)

        y.append(eph_stock_data.iloc[i+timesteps][target_column])

    X = np.array(X)
    y = np.array(y)

    # Создаем RNN модель
    input_shape = (X.shape[1], X.shape[2])
    model = Sequential()
    model.add(SimpleRNN(64, activation='tanh', input_shape=input_shape, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(SimpleRNN(32, activation='tanh', return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    model.compile(optimizer=Adam(), loss='mean_squared_error', metrics=['mae'])

    # Обучаем модель
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    model_checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True)
    callbacks = [early_stopping, model_checkpoint]

    model.fit(X, y, epochs=epochs, batch_size=batch_size, validation_split=0.2, callbacks=callbacks, verbose=1)

    return model

def get_stock_predict(trained_model, stock_data, target_column='value', timesteps=30):

    engine = create_engine(f"mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_SERVER}:{DB_PORT}/{DB_NAME}")


    moscow_timezone = pytz.timezone("Europe/Moscow")
    current_time = datetime.now(moscow_timezone)

    start_date = stock_data['date'].max() + pd.Timedelta("1 day")
    end_date = stock_data['date'].max() + pd.Timedelta("31 day")

    query = f"""
    SELECT *,
           CONCAT(year, '-', LPAD(month, 2, '0'), '-', LPAD(day, 2, '0')) AS full_date
    FROM astro
    WHERE CONCAT(year, '-', LPAD(month, 2, '0'), '-', LPAD(day, 2, '0')) BETWEEN '{start_date}' AND '{end_date}';
    """

    astro_data = pd.read_sql(query, engine)

    astro_data['date'] = pd.to_datetime(astro_data[['year', 'month', 'day']])

    stock_data['date'] = pd.to_datetime(stock_data['date'])
    merged_data = pd.merge(stock_data, astro_data, on='date', how='right')

    merged_data = merged_data.drop(columns=['date', 'full_date','value'], errors='ignore')

    scaler = StandardScaler()
    feature_columns = merged_data.columns.difference([target_column])

    merged_data[feature_columns] = scaler.fit_transform(merged_data[feature_columns])

    initial_window = merged_data.iloc[-timesteps:][feature_columns].values
    X_pred = np.array([initial_window])

    future_dates = [start_date + pd.Timedelta(f"{i} day") for i in range(timesteps)]
    predictions = []

    for i in range(30):

        pred_value = trained_model.predict(X_pred)[0][0]

        predictions.append(pred_value)

        new_row = np.zeros((1, X_pred.shape[2]))
        X_pred = np.concatenate([X_pred[:, 1:, :], new_row[np.newaxis, :, :]], axis=1)
        X_pred[0, -1, :] = np.append(X_pred[0, -1, :-1], pred_value)

    pred_value = stock_data['value'].iloc[-1]

    pred_list = []

    coeff = stock_data.dropna(subset=['value']).reset_index(drop=True)

    counter = 50 * 4**((-9) * len(coeff) * 10**(-4) +0.3)+0.15

    for i in range(timesteps):
      pred_value += predictions[i] * counter
      pred_list.append(pred_value)

    prediction_df = pd.DataFrame({
        'date': future_dates,
        'predicted_value': pred_list
    })

    return prediction_df

def calculate(tiker):

    stock_data = company_stock_history(tiker)
    eph_stock_data = merge_stock_with_astro(stock_data)
    trained_model = prepare_and_train_model(eph_stock_data)
    predicted_prices = get_stock_predict(trained_model, stock_data)

    return stock_data, predicted_prices

def convert_dates_to_strings(df):

    if 'date' in df.columns:
        df['date'] = df['date'].astype(str)
    return df

## Создание датафрейма для обучения модели

# Запуск NGROK для связывания гугл коллабов

In [ ]:
# Обработчик HTTP-запросов
!ngrok authtoken '2qaRAbcjuH5F6DqNpVIMLx2Nmj3_2MEDzKSDeWR4s39nrX6ZE'

import json
import logging

logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

class SimpleHandler(BaseHTTPRequestHandler):
    def do_GET(self):
        try:
            # Проверка на наличие тикера
            ticker = self.path[1:]
            if not ticker:
                self.send_response(400)
                self.end_headers()
                self.wfile.write(b"Ticker is required in the URL path.")
                logging.error("Тикер отсутствует в запросе.")
                return

            logging.info(f"Получен запрос для тикера: {ticker}")

            df1, df2 = calculate(ticker)

            df1 = convert_dates_to_strings(df1)
            df2 = convert_dates_to_strings(df2)

            response_data = {
                'df1': df1.to_dict(orient='records'),
                'df2': df2.to_dict(orient='records')
            }

            response_json = json.dumps(response_data)
            self.send_response(200)
            self.send_header('Content-Type', 'application/json')
            self.end_headers()
            self.wfile.write(response_json.encode())
            logging.info(f"Ответ отправлен: {response_json}")
        except Exception as e:

            logging.error(f"Ошибка обработки запроса: {e}")
            self.send_response(500)
            self.end_headers()
            self.wfile.write(f"Server error: {str(e)}".encode())

public_url = ngrok.connect(8071)
print(f"Public URL: {public_url}")

server_address = ('', 8071)
httpd = HTTPServer(server_address, SimpleHandler)
print("Server is running...")
httpd.serve_forever()

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Public URL: NgrokTunnel: "https://3cc7-34-56-131-214.ngrok-free.app" -> "http://localhost:8071"
Server is running...


<ipython-input-2-7994a854c919>:87: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  full_data = pd.concat(all_data, ignore_index=True)


Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


84/84 ━━━━━━━━━━━━━━━━━━━━ 5s 34ms/step - loss: 12174917.0000 - mae: 2840.3560 - val_loss: 44507532.0000 - val_mae: 6628.2974
Epoch 2/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - loss: 11854480.0000 - mae: 2807.8191 - val_loss: 44548220.0000 - val_mae: 6631.3765
Epoch 3/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - loss: 12013854.0000 - mae: 2809.3313 - val_loss: 44584980.0000 - val_mae: 6634.1592
Epoch 4/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - loss: 12243348.0000 - mae: 2855.1084 - val_loss: 44620236.0000 - val_mae: 6636.8252
Epoch 5/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 12285563.0000 - mae: 2828.1836 - val_loss: 44654696.0000 - val_mae: 6639.4307
Epoch 6/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - loss: 12526087.0000 - mae: 2848.3335 - val_loss: 44688736.0000 - val_mae: 6642.0039
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 252ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/ste

127.0.0.1 - - [01/Jan/2025 18:28:11] "GET /PHOR HTTP/1.1" 200 -
